In [1]:
# Lab 12 Character Sequence RNN
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility

sample = " i too like tobigs, too"
#uniqe한 것을 리스트 형태로
idx2char = list(set(sample))  # index -> char  
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex

sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0

cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(30):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(i, "loss:", l, "Prediction:", ''.join(result_str))


C:\Users\S9304\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.
0 loss: 2.6933239 Prediction: yoyvuvt tvuvtvtotvyvt totvt
1 loss: 2.361414 Prediction: yo  o t   o t  ut to t o u 
2 loss: 1.9565628 Prediction: yo loue  oout  ut  out onu 
3 loss: 1.5527928 Prediction: yoslone  Hout Hut Hout onet
4 loss: 1.2127416 Prediction: ye lone  yout dut youtdon't
5 loss: 0.9597146 Prediction: ye lone  Bou. du  youtdon't
6 loss: 0.7042582 Prediction: ye love  you. But you don't
7 loss: 0.5196396 Prediction: ye love  you. But you don't
8 loss: 0.3828514 Prediction: ye loves you  But you don't
9 loss: 0.2919817 Prediction: He loves you. But you don't
10 loss: 0.19665368 Prediction: He loves you. But you don't
11 loss: 0.12651816 Prediction: He loves you. But you don't
12 loss: 0.098257914 Prediction: He loves you. But you don't
13 loss: 0.06881092 Prediction: He loves you. But you don't
14 loss: 0.049330156 Prediction: He loves you. But you don't
15 loss: 0.033712592 Prediction: He loves you.

In [2]:
"I my me my two too like tobigs, too"

'I my me my two too like tobigs, too'